In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model

In [3]:
# fine tuning = 미세 조정(모델을)
# fine tuning을 위해선 pre-trained model이 필요
# pre trained 모델이란, 거대한 데이터를 미리 학습한 모델을 의미
base_model = tf.keras.applications.ResNet50(include_top=False, weights='imagenet')

94765736/94765736 [==============================] - 1s 0us/step


In [4]:
# Fine tuning 순서
# 1. base model은 모델 파라미터를 Freeze, Fully Connected Layer만 학습
# optimizer의 LR = Default

# 2-1. 우리의 데이터가 pre-trained 모델이 학습한 데이터보다 많을 때
# 2-1-1. 카테고리가 비슷하다 -> base model의 50%
# 2-1-2. 카테고리가 다르다   -> base model의 50~100%

# 2-2 우리의 데이터가 pre-trained 모델이 학습한 데이터보다 적을 때
# base model의 10~30% 만 업데이트
# optimizer의 LR = Default / 10 ~ Default / 100

for layer in base_model.layers:
  layer.trainable = False


In [ ]:
base_model.summary() # Freeze 확인

In [11]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
output_layer = Dense(1, activation = 'sigmoid')(x)
our_resnet_model = Model(inputs = base_model.input, outputs = output_layer)
our_resnet_model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, None, None, 3)]      0         []                            
                                                                                                  
 conv1_pad (ZeroPadding2D)   (None, None, None, 3)        0         ['input_1[0][0]']             
                                                                                                  
 conv1_conv (Conv2D)         (None, None, None, 64)       9472      ['conv1_pad[0][0]']           
                                                                                                  
 conv1_bn (BatchNormalizati  (None, None, None, 64)       256       ['conv1_conv[0][0]']          
 on)                                                                                        

In [12]:
our_resnet_model.compile('adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
!mkdir dataset
!unzip /content/drive/MyDrive/pokedata.zip -d dataset

In [17]:
train_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory(
    'dataset/',
    target_size = (224, 224),
    batch_size = 32,
    class_mode = 'binary'
)

Found 0 images belonging to 2 classes.
